# Set up

In [ ]:
using IntervalArithmetic, Combinatorics, Polynomials, Serialization, LinearAlgebra, Plots, LaTeXStrings

In [ ]:
include("quadrature.jl")

In [ ]:
# start here with precomputed vandermonde matrices
n = 1200

# approximate solution
Ū = interval.(deserialize("Ubar"));

# regularised Vandermonde matrices for calculating u\bar
V̄1 = deserialize("V1r")
V̄3 = deserialize("V3r")
# regularised Vandermonde matrices for 2p and p+1 products
V̄8 = deserialize("V8r")
V̄10 = deserialize("V10r")
p = interval(5//3)
d = 2
α = d//2
Z = interval(4)*interval(BigFloat, π)

# m = n+1, (n+1)ᵗʰ eigenvalue
λₘ = interval(d//2+n+1)
λ₀ = interval(d//2);

We compute $\bar{u}$ by projection, i.e.

$$\bar{u}_n = \int_{\mathbb{R}^d} \bar{U}^3\hat{\psi}_n\mu\mathrm{d} x$$

In [ ]:
# Define ū
ū = V̄1'*((V̄3*interval(Ū)).^3);

In [ ]:
𝔏 = Diagonal(interval.(BigFloat,collect(0:n)).+λ₀);

In [ ]:
LinearAlgebra.norm(v::Vector) = sqrt(sum(v.^2))

# computes Lᵖ/H¹ norms before taking fractional exponent
function L2(u)
    return sum(u.^2)
end

function H2(u)
    return sum((𝔏*u).^2)
end

function L103(u)
    return sum(cbrt.(V̄10*u).^10)
end

# rigorous upper bound of the 2-norm of a matrix
function op_norm(A)
    if size(A) == (2,2)
        Z = sqrt(sum(A.^2) + sqrt(((A[1,2]+A[2,1])^2+(A[1,1]-A[2,2])^2)*((A[1,2]-A[2,1])^2+(A[1,1]+A[2,2])^2)))/sqrt(interval(2))
        if isguaranteed(Z)
            return interval(sup(Z))
        else
            return Z
        end
    else
        all(isguaranteed.(A)) || error("matrix not guaranteed")
        return sqrt(interval(maximum(sup.(sum(abs.(A), dims = 1))))*interval(maximum(sup.(sum(abs.(A), dims = 2)))))
    end
end

# Proof

In [ ]:
# rigorous L²-norms of ūᵖ⁻¹ψ̂ⱼ
int = sum((V̄10.*(cbrt.(V̄10*ū).^2)).^2, dims = 1);

In [ ]:
# Gram matrix induced by ūᵖ⁻¹
G = interval.(Float64, V̄8)'*(interval.(Float64, cbrt.(V̄8*ū).^2).*interval.(Float64, V̄8));

In [ ]:
# PₙDF(ū)Pₙ
DFū = interval.(Float64, interval(I(n+1)) - inv(𝔏)+ p*𝔏\G);

In [ ]:
# approximate numerical inverse of PₙDF(ū)Pₙ
Aₙ = interval.(BigFloat,inv(mid.(DFū)));

In [ ]:
# PₙF(ū)
PFū = ū -inv(𝔏)*(ū/(p-interval(BigFloat,1))-V̄8'*(cbrt.(V̄8*ū).^5));

In [ ]:
# Y = interval(sqrt(H1(Aₙ*PFū)+abs(L4(ū)-L2((V3'*(W.*(V3*ū).^2))*interval(big(2))^(-3//2)))/λₘ).hi)
Y = interval(sup(sqrt(H2(Aₙ*PFū)+abs(L103(ū)-L2(V̄8'*(cbrt.(V̄8*ū).^5))))))

In [ ]:
supū = sum(abs.(ū));

In [ ]:
w = sqrt.(abs.([int[i] - L2(G[i,:]) for i=1:n+1]));

In [ ]:
Z¹¹ = op_norm(𝔏*(interval(I(n+1)) - interval.(Float64, Aₙ)*interval.(Float64, DFū))*inv(𝔏));

In [ ]:
Z¹² = p*norm(abs.(𝔏*Aₙ*inv(𝔏))*w)/λₘ;
Z²¹ = p*norm(𝔏\w)
Z²² = interval(sup((interval(1)/(p-interval(1))+p*supū)/λₘ));

In [ ]:
Z₁ = op_norm([Z¹¹ Z¹² ; Z²¹ Z²²])

In [ ]:
if sup(Z₁)>=1
    println("ū cannot be validated")
end

In [ ]:
[Z¹¹ Z¹² ; Z²¹ Z²²]

In [ ]:
C₀ = I"0.56419"
C₁ = I"0.79789"
C₂ = I"0.23033"
# C(2)
C2 = sqrt(Z)*(interval(2//d)*C₀ + sqrt(interval(2//d))*C₁ + C₂)

In [ ]:
Z₅₃ = C2^interval(2//3)*interval(max(sup(op_norm(𝔏*Aₙ*inv(𝔏))), 1))*interval(5//3)

In [ ]:
Q(δ) = Z₁ + Z₅₃*δ^interval(2//3)
P(δ) = Y + Z₁*δ + interval(3//5)*Z₅₃*δ^interval(5//3)-δ

In [ ]:
δ̲ = Y/(interval(1)-Z₁)*(interval(1)+interval(BigFloat,2)^(-46))

In [ ]:
if sup(P(δ̲))<0 && sup(Q(δ̲))<1
    println("δ̲ validated")
end

In [ ]:
δ̄ = ((interval(1)-Z₁)/Z₅₃)^interval(3//2)*(interval(1)-interval(BigFloat,2)^(-50))

In [ ]:
if sup(P(δ̄))<0 && sup(Q(δ̄))<1
    println("δ̄ validated")
end

# Checking positivity

In [ ]:
# infinity norm error
inf_err = C2*δ̲

In [ ]:
m = 10
supm = inf_err+sum(abs.(ū[m+2:end]))
Pₘ = zeros(Interval{BigFloat},m+1)
for j = 0:m
#     println(j)
    Lⱼ =  [interval((-1)^i)*interval(BigFloat, binomial(j,i)//factorial(i)) for i=0:j]
    Pₘ[1:j+1] += ū[Int64(j+1)]*Lⱼ
end

In [ ]:
r₀ = interval(BigFloat,5.0)
z = mince(interval(0,r₀.^2/interval(4)), 2000)

if all(inf.(evalpoly.(z,tuple(Pₘ)).*exp.(-z).-supm.*exp.(-z/interval(2))).>0) && inf(interval(d//4)-interval(1)/(p-interval(1))+r₀^2/interval(16)-((supū+inf_err)*exp(-r₀^2/interval(8)))^(p-interval(1)))>0
    println("positivity checked")
else
    println("positivity not checked")
end

# Plotting

In [ ]:
rloc = collect(0:600)/100
zloc = interval.(big.(rloc.^2/4))
sol = zeros(Interval{BigFloat},size(zloc));

In [ ]:
setprecision(4096)
Lprev = ones(Interval{BigFloat}, length(zloc)) 
L = Lprev .- zloc
sol += ū[Int64(1)]*Lprev
for m=1:big(n)
    sol += ū[Int64(m+1)]*L
    Lhold = copy(L)
    L[:] = ((interval(BigFloat,2*m+1) .-zloc).*L .- interval(BigFloat,m)*Lprev)/interval(BigFloat, m+1)
    Lprev[:] = Lhold
end

In [ ]:
plot(rloc, Float64.(mid.(sol.*exp.(-zloc))), xlabel = L"r", ylabel = L"\bar{u}", legend = false, dpi =1000)

In [ ]:
png("fractional_sol")